In [0]:
%sql

CREATE VOLUME IF NOT EXISTS movies_catalog.landing.landing_zone_movies;
CREATE VOLUME IF NOT EXISTS movies_catalog.bronze.bronze_zone_movies;
CREATE VOLUME IF NOT EXISTS movies_catalog.silver.silver_zone_movies;
CREATE VOLUME IF NOT EXISTS movies_catalog.gold.gold_zone_movies;


In [0]:
# Notebook: 1_landing_to_bronze

from pyspark.sql import SparkSession
from delta.tables import *

spark = SparkSession.builder.appName("Landing_to_Bronze").getOrCreate()

landing_path = "/Volumes/movies_catalog/landing/landing_zone_movies"
bronze_path  = "/Volumes/movies_catalog/bronze/bronze_zone_movies"

dbutils.fs.mkdirs(bronze_path)

files = [
    "movies_casts.csv",
    "movies_meta.csv",
    "movies_ratings.csv",
    "movies_revenue.csv",
    "movies_top100.csv"
]

for file in files:
    print(f"📥 Lendo arquivo da Landing: {file}")
    
    df = (
        spark.read
            .option("header", True)
            .csv(f"{landing_path}/{file}")
    )

    
    for c in df.columns:
        df = df.withColumnRenamed(
            c,
            c.strip()
             .replace(" ", "_")
             .replace("(", "")
             .replace(")", "")
             .replace("%", "pct")
        )

    table_name = file.replace(".csv", "")
    output_path = f"{bronze_path}/{table_name}"

   
    (
        df.write
          .format("delta")
          .mode("overwrite")
          .option("mergeSchema", "true")
          .save(output_path)
    )

    print(f"✅ {file} movido para Bronze com sucesso → {output_path}")
